In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *

In [3]:
from matplotlib import pyplot as plt

In [4]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [5]:
aoi = GERD_aoi

In [16]:
SAR = SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filterDate('2022-01-01','2022-03-10')\
    .select(['VV','VH']).median()

In [17]:
# Map.add_layer_widgets(SAR,{'min':-25,'max':5,'bands':['VH']},'SAR2')

In [18]:
# SAR = ee.Image(toDB(RefinedLee(toNatural(SAR)))).rename('VH_Filtered')

In [19]:
histogram =  SAR.select('VH').reduceRegion(
                                
            reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
            geometry = aoi, 
            scale = 10,
            bestEffort = True  
)

In [20]:
histogram.getInfo()

{'VH_histogram': {'bucketMeans': [-39.88531110644547,
   -39.625,
   -39.375,
   -39.03305842801515,
   -38.80725057510129,
   -38.57539967552705,
   -38.39969513002522,
   -38.11725643886808,
   -37.92145032996693,
   -37.56869272500075,
   -37.363527390948256,
   -37.11129891938266,
   -36.86497609775875,
   -36.62570415030611,
   -36.37179551503006,
   -36.111102655076316,
   -35.86865816246705,
   -35.6178784871157,
   -35.3653710976557,
   -35.116908639509,
   -34.868663813640964,
   -34.617796713655366,
   -34.368166760946146,
   -34.11941099284976,
   -33.86747617584201,
   -33.61876756251901,
   -33.36746164511716,
   -33.119221138199,
   -32.86764544950329,
   -32.61854953320076,
   -32.36935715374651,
   -32.12031966457634,
   -31.870974459688007,
   -31.62164404772855,
   -31.372663883492585,
   -31.12341246873769,
   -30.875040292234477,
   -30.625679250444147,
   -30.376532428787282,
   -30.1275871792233,
   -29.878575795369688,
   -29.628375566501607,
   -29.3790525331188

In [14]:
threshold = otsu(histogram.get('VH_histogram'))

In [15]:
threshold.getInfo()

-22.622093910059853

In [11]:
water = SAR.select('VH').lt(threshold).selfMask().clip(aoi)

In [12]:
x = water.connectedPixelCount(1024)

In [13]:
rem = x.lt(1024)

In [14]:
m = rem.Not()

In [15]:
water = water.updateMask(m)

In [16]:
feature = water.reduceToVectors(
           geometry = water.geometry(),
           crs ='EPSG:32636',
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = True,
           labelProperty =  'water_cover',
           bestEffort = True
          )

In [17]:
elevation_dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')\
           .filter(ee.Filter.bounds(aoi))\
           .select('DSM')\
           .median()\
           .clip(aoi)\
           .reproject(crs = 'EPSG:32636', scale = 30 )

In [18]:
lake_dem = elevation_dataset.clip(feature)

In [24]:
max_ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = feature.geometry(), 
                         scale = 30, 
                         crs = 'EPSG:32636',
                        maxPixels = 1e11).get('DSM')

In [25]:
max_ele.getInfo()

613